<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/coordinates_display_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Display box coordinates resulting from object detection or image augmentation on images
---
*Last Updated 1 June 2021*   
Use this notebook to verify the coordinates resulting from either object detection or image augmentation. 

__Coordinates from object detection__: Bounding boxes resulting from object detection were exported from [taxon]_generate_crops_yolo.ipynb or [taxon]_generate_crops_tf2.ipynb. Crop coordinates were exported to [taxon]_crops_[yolo or tf2]_1000img_display_test.tsv.

__Coordinates from pre-processing and augmentation__: Bounding boxes resulting from pre-processing and augmentation of EOL user-generated cropping coordinates and images were tidied and exported from [taxon]_preprocessing.ipynb to [taxon]_crops_train_aug_fin.tsv and [taxon]_crops_test_notaug_fin.tsv. If results aren't as expected, data reformatting and tidying steps in [taxon]_preprocessing.ipynb file can be adjusted and results re-displayed accordingly.

## Installs & Imports
---


In [ ]:
# Mount Google Drive to import your file containing coordinates (object detection bounding boxes, cropping coordinates, etc.)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# For importing data and images
import os
import numpy as np 
import pandas as pd
import urllib
from urllib.request import urlretrieve
from six.moves.urllib.request import urlopen
from six import BytesIO
from collections import defaultdict
from io import StringIO
from IPython.display import display

# For drawing on and displaying images
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import cv2
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

# For saving images
# Un-comment out if running "Save crop dimensions displayed on images to Google Drive" below
!pip install scipy==1.1.0
import scipy
from scipy import misc

## Display crop dimensions on images
---

In [ ]:
# Define functions

# Read in data file exported from "Combine output files A-D" block above
def read_datafile(fpath, sep="\t", header=0, disp_head=True):
    """
    Defaults to tab-separated data files with header in row 0
    """
    try:
        df = pd.read_csv(fpath, sep=sep, header=header)
        if disp_head:
          print("Data header: \n", df.head())
    except FileNotFoundError as e:
        raise Exception("File not found: Enter the path to your file in form field and re-run").with_traceback(e.__traceback__)
    
    return df

# Draw cropping box on image
def draw_box_on_image(df, img):
    # Get box coordinates
    xmin = df['xmin'][i].astype(int)
    ymin = df['ymin'][i].astype(int)
    xmax = df['xmin'][i].astype(int) + df['crop_width'][i].astype(int)
    ymax = df['ymin'][i].astype(int) + df['crop_height'][i].astype(int)
    boxcoords = [xmin, ymin, xmax, ymax]

    # Set box/font color and size
    maxdim = max(df['im_height'][i],df['im_width'][i])
    fontScale = maxdim/600
    box_col = (255, 0, 157)
  
    # Add label to image
    tag = df['class_name'][i]
    image_wbox = cv2.putText(img, tag, (xmin+7, ymax-12), cv2.FONT_HERSHEY_SIMPLEX, fontScale, box_col, 2, cv2.LINE_AA)  
  
    # Draw box label on image
    image_wbox = cv2.rectangle(img, (xmin, ymax), (xmax, ymin), box_col, 5)

    return image_wbox, boxcoords

# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im_h, im_w = image.shape[:2]
  
    return image

In [ ]:
# Display crop dimensions on images

# Import your file with cropping coordinates
# TO DO: Enter filepath in form field
fpath = "/content/drive/MyDrive/train/tf2/results/chiroptera_cropcoords_tf2_rcnn_concat_displaytest.tsv" #@param {type:"string"}
df = read_datafile(fpath)

# TO DO: Choose start index in cropping dataframe
start = 0 #@param {type:"integer"}
# TO DO: Choose how many images to display coords on (max. 50)
num_images = 5 #@param {type:"slider", min:0, max:50, step:5}
stop =  start + num_images

# Display cropping dimensions on images
print("\nDisplaying cropping coordinates on images: \n")
for i, row in df.iloc[start:stop].iterrows():
    # Read in image 
    url = df['eolMediaURL'][i]
    img = url_to_image(url)
  
    # Draw bounding box on image
    image_wbox, boxcoords = draw_box_on_image(df, img)
  
    # Plot cropping box on image
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(image_wbox)

    # Display image URL and coordinates above image
    # Helps with fine-tuning data transforms in post-processing steps above
    plt.title('{}) {} \n xmin: {}, ymin: {}, xmax: {}, ymax: {}'.format(i+1, url, boxcoords[0], boxcoords[1], boxcoords[2], boxcoords[3]))

## Save crop dimensions displayed on images to Google Drive
---
Useful if want to share results with someone remotely. Saves images with detection boxes to Google Drive.

In [ ]:
# Display crop dimensions on images & save results

# Import your file with cropping coordinates
# TO DO: Enter filepath in form field
fpath = "/content/drive/MyDrive/train/tf2/results/chiroptera_cropcoords_tf2_rcnn_concat_displaytest.tsv" #@param {type:"string"}
df = read_datafile(fpath)

# Path to folder for exporting images with bounding boxes
# TO DO: Enter path to where you want images saved to in form field
pathbase = "/content/drive/MyDrive/train/tf2/out/" #@param {type:"string"}

# TO DO: Choose start index in cropping dataframe
start = 0 #@param {type:"integer"}

# TO DO: Choose how many images to display coords on (max. 50)
num_images = 5 #@param {type:"slider", min:0, max:50, step:5}
stop =  start + num_images

# Display cropping dimensions on images
print("\nDisplaying cropping coordinates on images: \n")
for i, row in df.iloc[start:stop].iterrows():
    # Make output path
    path = pathbase + str(df.dataObjectVersionID[i]) + '.jpg'

    # Read in image 
    url = df['eolMediaURL'][i]
    img = url_to_image(url)
  
    # Draw bounding box on image
    image_wbox, boxcoords = draw_box_on_image(df, img)
  
    # Plot cropping box on image
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(image_wbox)

    # Display image URL and coordinates above image
    # Helps with fine-tuning data transforms in post-processing steps above
    plt.title('{}) Image from: {} \n Saved to: {}'.format(i+1, url, path))

    # Export image to Google Drive
    misc.imsave(path, image_wbox)